# Javaで推論を実行する

## コンパイルする
mavenでコンパイルする

In [1]:
!mvn clean antrun:run

[INFO] Scanning for projects...
[INFO] 
[INFO] --------------------< jp.go.digital.kanjikana:java >--------------------
[INFO] Building java 1.1
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- clean:3.2.0:clean (default-clean) @ java ---
[INFO] 
[INFO] --- antrun:3.1.0:run (default-cli) @ java ---
[INFO] No Ant target defined - SKIPPED
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  1.010 s
[INFO] Finished at: 2025-01-21T15:31:04+09:00
[INFO] ------------------------------------------------------------------------


## javaで推論を実行する